# Module2 Summary

## Introduction and Background information:

Yelp is an American Internet corporation. It mainly hosts Yelp.com and the Yelp mobile app, which publish crowd-sourced reviews about local businesses. To help people find great local businesses, Yelp has provided a platform where all users are able to write reviews of each business.

This dataset contains several different dimensions. We are trying to use this dataset to perform a sentiment analysis for each review. Basically, we have two main goals:
1.	Find out what makes a review positive or negative.
2.	Predict a review’s rating based on given features(mainly the text). 


## Thesis statement:

From our analysis, whether a review is positive or negative depends on review date, category of the restaurant, whether a restaurant is chain or local, type of food, quality of service, price, and atmosphere. We tried several models, such as multinomial naive bayes, lasso, and svr to make predictions of rating stars, and it turned out that nerual network had the lowest root mean squared error after dimension reduction using word2vec.

## Data Process:

First, take a short look at dataset:
<img src="https://github.com/Sommerzzz/yelp_star/blob/master/image/data.png?raw=true" height="600" width="1000">

Then, look into each feature in details. 

###      Name

For restaurants’ names, we first notice that there are some chain restaurants’ labels such as McDonald’s, Dairy Queen. This fact reminds us that rating for chain restaurants and local restaurants may be different. To find out if this true or not, we trying to do the U-test and the p-value is nearly to 0. Thus, we have strong evidence to say that this ‘chain and local’ label has some effects on star rating. We consider it as a feature in our models.

###       Time:

From plot of star mean per month, we could know that with the increase of number of reviews, star mean goes to be stable and after time series analysis, we could not find any seasonal trend or other special trend.
Since the first part with high star mean only contains about 5% of the whole data points, and the difference between this part and the rest of our data points is not too much large, we just ignore this issue and drop Time feature in our dataset.

###       City and Location(latitude and longitude):

Since city names and locations provide almost same information, we consider them together. First, look at the data distribution. We make a points plot with latitude as y-axis and longitude as x-axis. There are three clusters on the map: around Las Vegas, around Toronto and London(Notice that there are some points located in Asia which are really mistakes). The distriubtion reminds us that we can use K-means to cluster location. The cluster labels may provide us with some useful infomation for modeling.
 
###       Categories:

After using strsplit function in R and more cleaning process, we got 605 unique category names. Some of them show skewness, while others show approximately uniform distribution among 1-5 stars. We generate a sparse matrix with 1.5 million rows and 605 coloums, and may consider this feature in our final model.

###       Text:

We first explored review texts, and found that there are 34 unique languages. Since 99% of them are English, we just ignore other languages.

We did the following steps to preprocess review text.

(1) We created a special marks feature baesd on texts to see whether a review contains symbols like '!!', '?!', '!?', ':)', ':-)', ':(', ':-('. Because these special symbols give strong sentiment information, they will be useful in modeling. 

(2) Then, all texts are lower-cased, and punctuations are removed for simplicity. 

(3) After above steps, we split texts and counted frequency of each word. Words that appear less than 20 times are removed. In this step, other languages are removed. We also selected words of high frequency but are meaningless manually, and removed them.

(4) For different models, we used different methods to deal with text data:
1. For multinomial naive bayes, lasso, and svr, we created count sparse matrix and TF-IDF sparse matrix;
2. For neural network, we first used word2vec to convert each word to a vector of numbers, and then calculated mean and standard deviation for each position for each review.

###       Train-test split

We randomly split 80% train data as our training set, and 20% as our test set to validate our models.

## Modeling and Model selection:

### **Rating Modeling**
#### Linear Model:
Since we have converted all text reviews into numerical values and linear model is simple and fast, try linear model as our first model is quite reasonable. Mainly, we use the Lasso method. 

Features we used are: 1. count sparse matrix and tfidf sparse matrix; 2. the text length for each review; 3. the 'Chain or Local' label; 4. the 'special mark' label.

#### Naive Bayes:
Since we have a quite large dataset, we consider using Multinomial Naive Bayes to do modeling. Naive Bayes is a simple probabilistic classifiers. It uses very simple assumptions and often feedback quite good results. Features we used are the same as lasso model.

#### Support Vector Regression:
We consider use the SVR to do the modeling because it shows a good performance in other's nlp research. SVR is a supervised learning model for regression problem. An SVR model is a representation of the examples as points in space, mapped so that the examples of the separate categories are divided by a clear gap that is as wide as possible. Features we used are the same as lasso model.


#### Nerual Network:
At last, we try to use the Neural Network since this is a common and effective model in natural language processing. We used BP neural network.

Features we used are: 1. mean vector given by word2vec; 2. std vector given by word2vec; 3. text length for each review.

### Toptics Modeling:
To find what makes a review positive or negative, we try to conduct topic modeling here. Latent Dirichlet allocation is a generative statistical model that can extract some common topics from dataset. Followings are the setps of our LDA modeling:
1. we group 1 and 2 stars' reviews as the Negative reviews, 4 and 5 stars' reviews as Positive reviews.
2. For each group, we try LDA with different parameters.
After serval rounds of LDA modeling, we summary the result in a figure.
<img src="https://github.com/Sommerzzz/yelp_star/blob/master/image/lda.png?raw=true" height="400" width="600">

### Model selection and diagnosis

**Support vector machine**: It takes a long time to implementation with all the data and accuracy is not satisfied.

**Linear model with Lasso**: Due to feature selection limitation, the outcome is not satisfied.

**Multinomial Naive bayes**: The model is fast and gives us a relatively lower mean square error compared with linear model and svr, but worse than neural network.

**Neural Network**: This method has relatively high computing speed and relatively high predictive accuracy.
The process of parameters tuning is in the following table:




|                                                                         | MSE    | RMSE | ACC    | Test- Mse  |
| ----------------------------------------------------------------------- | ------ | ---- | ------ | ---------- |
| Mean NNT ,Len and  Specmark( two hidden layers)                         | 0.5540 | 0.75 | 0.5560 | 0.56378414 |
| Mean NNT  and Min NNT, Len ,Specmark, chain, cluster(one hidden layers) | 0.5419 | 0.78 | 0.5598 | 0.52620397 |
| Mean NNT  and Min NNT, Len ,Specmark, chain, cluster(two hidden layers) | 0.5276 | 0.77 | 0.5677 | 0.56346143 |
| Mean NNT and Std NNT, len (with two hidden layers)                      | 0.46   | 0.67 | 0.5990 | 0.50478211 |

The comparison of best mse for each method are shown in the following table:

| Model Name | Multinomial Naive | Lasoo | SVM  | Neural Network |
| ---------- | ----------------- | ----- | ---- | -------------- |
| Mse        | 0.79              | 1.37  | 1.45 | 0.45           |

## Summary:

1. After trying several reasonable models, we found out that neural network with 200-dimension Word2Vec as the input layer performs best.
2. Due to limitation of compute performance, we could not add categories sparse matrix into our final model.
3. After trying mean, median, minimum, and maximum of each entry of vectorlized review text as the input layer, we found that mean performs best.

### Final results:
**(1)** Our final model is NNT mdoel, the performance are:

| Mse on validation set | Accuracy on validation set | Rmes on Kaggle |
| --------------------- | -------------------------- | -------------- |
| 0.450                 | 0.60                       | 0.671          |

**(2)** To answer the question what makes a review positive or negative, after conisderation, we decide to use our predictions. The main reason is that the result comes from LDA takes too much objective choices which can be quite confusing sometimes. Using our model, if the predicted star is 1 or 2, we consider the review as negative. If the predicted star is 3,4 or 5, we consider the review as positive.

### Contributions:

**Wenyan Zhou**: *Data processing, clean text, topic modeling and github repo.*

**Bolaing Ma**: *Clean train text data, word to vector model.*

**Yan Chu**: *Category manipulation, data visulization,power point making.

**Ruidong Lan**: *Coding, modeling and summary for each model.*


## Reference:
[1]DATAQUEST. 2015. *Natural Language Processing with Python Tutorial.* [ONLINE] Available at: https://www.dataquest.io/blog/natural-language-processing-with-python/. [Accessed 1 May 2015].

[2]Wikipedia. 2018. *Hierarchical clustering.* [ONLINE] Available at: https://en.wikipedia.org/wiki/Hierarchical_clustering. [Accessed 26 February 2018].

[3]David Robinson. 2016. *Does sentiment analysis work? A tidy analysis of Yelp reviews.* [ONLINE] Available at: http://varianceexplained.org/r/yelp-sentiment/. [Accessed 21 July 2016].

[4]RPubs. 2016. *LDA Topic Models.* [ONLINE] Available at: https://rpubs.com/Zyrix/yelptask1. [Accessed 27 July 2016].
